# Simple example of time domain beamforming

Let's imagine that we have a point sound source in the far field, which is effectively creating planar waves impinging upon a two element array.

The situation is depicted in the figure below.

We begin by taking the [Jupyter Interactors](https://github.com/bokeh/bokeh/tree/2.3.0/examples/howto/notebook_comms/Jupyter%20Interactors.ipynb) example [User guide](https://docs.bokeh.org/en/latest/docs/user_guide/jupyter.html) and modifiying it to suit our requirements.

Note that this interact does not seem to work in JupyterLab 3 at the moment.


In [ ]:
from ipywidgets import interact
import numpy as np

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
output_notebook()

In [ ]:
x = np.linspace(0, 2*np.pi, 2000)
y = np.sin(x)

In [ ]:
p = figure(title="simple line example", plot_height=300, plot_width=600, y_range=(-5,5),
           background_fill_color='#efefef')
r = p.line(x, y, color="#8888cc", line_width=1.5, alpha=0.8)

In [ ]:
def update(f, w=1, A=1, phi=0):
    if   f == "sin": func = np.sin
    elif f == "cos": func = np.cos
    r.data_source.data['y'] = A * func(w * x + phi)
    push_notebook()

In [ ]:
show(p, notebook_handle=True)

In [ ]:
interact(update, f=["sin", "cos"], w=(0,50), A=(1,10), phi=(0, 20, 0.1))